In [4]:
import pandas as pd
import numpy as np
import joblib
import json

In [5]:
# --- Cargar el Modelo FINAL y los Datos CONTEXTUALES ---
MODEL_PATH = '../models/lgbm_final_context_model.pkl'
DATA_PATH = '../data/processed/contextual_featured_dataset.csv'

model = joblib.load(MODEL_PATH)
df = pd.read_csv(DATA_PATH)


In [6]:
# --- Preparar los Datos para la Predicción ---
# Usamos las columnas guardadas para asegurar la consistencia
COLUMNS_PATH = '../models/model_columns.json'
with open(COLUMNS_PATH, 'r') as f:
    model_columns = json.load(f)

X = df[model_columns]

In [7]:
# --- Hacer Predicciones y Calcular Errores ---
preds_log = model.predict(X)
df['predicted_value'] = np.expm1(preds_log)
df['error'] = df['predicted_value'] - df['market_value_in_eur']
df['abs_error'] = abs(df['error'])

In [8]:
# --- La Prueba de Fuego: Los 10 Mayores Errores Absolutos (¿Han mejorado?) ---
top_10_worst_errors = df.sort_values(by='abs_error', ascending=False).head(10)

print("\n--- Top 10 Mayores Errores Absolutos (Modelo FINAL con Contexto) ---")
display_cols = ['name', 'position', 'age', 'market_value_in_eur', 'predicted_value', 'error']
top_10_worst_errors[display_cols].style.format({
    'market_value_in_eur': "€{:,.0f}",
    'predicted_value': "€{:,.0f}",
    'error': "€{:,.0f}"
})


--- Top 10 Mayores Errores Absolutos (Modelo FINAL con Contexto) ---


,name,position,age,market_value_in_eur,predicted_value,error
3561,Vinicius Junior,Attack,24.459959,"€200,000,000","€82,946,600","€-117,053,400"
5299,Florian Wirtz,Midfield,21.634497,"€140,000,000","€25,746,363","€-114,253,637"
3221,Kylian Mbappé,Attack,26.250513,"€170,000,000","€59,783,400","€-110,216,600"
5730,Pedri,Midfield,22.318960,"€120,000,000","€34,485,718","€-85,514,282"
3410,Rodri,Midfield,28.484600,"€130,000,000","€46,881,878","€-83,118,122"
5111,Cole Palmer,Midfield,22.614648,"€130,000,000","€49,351,416","€-80,648,584"
5200,Jude Bellingham,Midfield,21.497604,"€180,000,000","€102,310,331","€-77,689,669"
3781,Moussa Diaby,Attack,24.889802,"€55,000,000","€128,838,279","€73,838,279"
3906,Phil Foden,Attack,24.804928,"€130,000,000","€56,390,121","€-73,609,879"
3412,Declan Rice,Midfield,25.921971,"€110,000,000","€38,603,866","€-71,396,134"
